In [1]:
"""
EN to ZN
1. Use GPT2-medium, Decoder based, pretrained with EN text and has 24 layers of transformer layers. On top of it is a projection layer with shape(768, 50257) and softmax layer
2. It was pretrained by EN text, so basically it can not handle EN to CN task without fine-tuning
3. Need a tokenizer that incorperate EN and CN, the embedding size and dictionary size are (768, 119,547)
4. Fine-tune, may need prompt + EN,CN pair.
5. The FFN consists of two linear transformations with weights of shapes (768, 3072) and (3072, 768)
6. Don't need mask in decoder only model
7. No CLS token in decoder based model, conventionally, we pick up the last token of each output Nth (, embedding dimension) so if the intput lenghth is 10 runs (assuming we have 10 input length, prompt (EN) + ZH tokens + newly generated EN tokens),
   and the maximum length is 20, we would go through the decoder model iteratively 10 more times or until meets the EOS token.
8. Each layer of transformer would pass down output shape (batch, seq, embedding dimension), and we can pick up the last token and get   to know how the representation evolved.
9. For the intermediate weights of FFN of each layers of transformer, it has representation shape of  (,768) and we can also put it into the comparison to the last token of the final transformer layer.

"""

"\nEN to ZN\n1. Use GPT2-medium, Decoder based, pretrained with EN text and has 24 layers of transformer layers. On top of it is a projection layer with shape(768, 50257) and softmax layer\n2. It was pretrained by EN text, so basically it can not handle EN to CN task without fine-tuning\n3. Need a tokenizer that incorperate EN and CN, the embedding size and dictionary size are (768, 119,547)\n4. Fine-tune, may need prompt + EN,CN pair.\n5. The FFN consists of two linear transformations with weights of shapes (768, 3072) and (3072, 768)\n6. Don't need mask in decoder only model\n7. No CLS token in decoder based model, conventionally, we pick up the last token of each output Nth (, embedding dimension) so if the intput lenghth is 10 runs (assuming we have 10 input length, prompt (EN) + ZH tokens + newly generated EN tokens),\n   and the maximum length is 20, we would go through the decoder model iteratively 10 more times or until meets the EOS token.\n8. Each layer of transformer would p

Package installation and importation

In [1]:
import os
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from pprint import pprint
from IPython.display import clear_output
from transformers import BertTokenizer, GPT2LMHeadModel
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Dropout, Conv1D, GlobalMaxPooling1D, Concatenate, Activation
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from transformers import logging
logging.set_verbosity_error()
import sklearn as sk
import os
from nltk.data import find
import matplotlib.pyplot as plt
import re
from transformers import TFGPT2LMHeadModel, GPT2Config
import tensorflow as tf

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-fad25e12-f839-584b-8e8a-7cb78730d793)


In [4]:
#check out the data source we have
tmp_builder = tfds.builder("wmt19_translate/zh-en")
pprint(tmp_builder.subsets)

{Split('train'): ['newscommentary_v14',
                  'wikititles_v1',
                  'uncorpus_v1',
                  'casia2015',
                  'casict2011',
                  'casict2015',
                  'datum2015',
                  'datum2017',
                  'neu2017'],
 Split('validation'): ['newstest2018']}


In [16]:
#download data by tfds.builder
config = tfds.translate.wmt.WmtConfig(
  version=tfds.core.Version('0.0.3'),
  language_pair=("zh", "en"),
  subsets={
    tfds.Split.TRAIN: ["newscommentary_v14"]
  }
)
builder = tfds.builder("wmt_translate", config=config)

# Download and prepare the dataset
builder.download_and_prepare()

# Set builder to dataset (data pipeline type), split it into training, validation, testing
examples = builder.as_dataset(
    split=['train[:10%]', 'train[10%:11%]', 'train[11%:12%]'],
    as_supervised=True
)

train_data, val_data, test_data = examples

train_examples, train_labels = next(iter(train_data.batch(5000)))
val_examples, val_labels = next(iter(val_data.batch(500)))
test_examples, test_labels = next(iter(test_data.batch(10)))

In [6]:
train_examples[:4]

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'The fear is real and visceral, and politicians ignore it at their peril.',
       b'In fact, the German political landscape needs nothing more than a truly liberal party, in the US sense of the word \xe2\x80\x9cliberal\xe2\x80\x9d \xe2\x80\x93 a champion of the cause of individual freedom.',
       b'Shifting to renewable-energy sources will require enormous effort and major infrastructure investment.',
       b'In this sense, it is critical to recognize the fundamental difference between \xe2\x80\x9curban villages\xe2\x80\x9d and their rural counterparts.'],
      dtype=object)>

In [17]:
#set up the input as prompt + EN and target label as ZH
#tokenized it using the tokenizer that can both handle EN and ZH

train_examples_str = [x.decode('utf-8') for x in train_examples.numpy()]
train_labels_str = [x.decode('utf-8') for x in train_labels.numpy()]
val_examples_str = [x.decode('utf-8') for x in val_examples.numpy()]
val_labels_str = [x.decode('utf-8') for x in val_labels.numpy()]
test_examples_str = [x.decode('utf-8') for x in test_examples.numpy()]
test_labels_str = [x.decode('utf-8') for x in test_labels.numpy()]



# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2-medium", pad_token_id=tokenizer.eos_token_id)
tokenizer.pad_token = tokenizer.eos_token

# Add a prompt to each input example
prompt = "Translate English to Chinese: "
input_texts_train = [prompt + text for text in train_examples_str]

# Tokenize inputs and labels
input_train_tokenized = tokenizer(input_texts_train, return_tensors='tf', padding='max_length', truncation=True, max_length=100)
label_train_tokenized = tokenizer(train_labels_str, return_tensors='tf', padding='max_length', truncation=True, max_length=100)


input_texts_val = [prompt + text for text in val_examples_str]
# Tokenize inputs and labels
input_val_tokenized = tokenizer(input_texts_val, return_tensors='tf', padding='max_length', truncation=True, max_length=100)
label_val_tokenized = tokenizer(val_labels_str, return_tensors='tf', padding='max_length', truncation=True, max_length=100)

In [8]:
# For training data
gpt_train_inputs = {
    'input_ids': input_train_tokenized['input_ids'],
    'attention_mask': input_train_tokenized['attention_mask']
}

# For validation data
gpt_val_inputs = {
    'input_ids': input_val_tokenized['input_ids'],
    'attention_mask': input_val_tokenized['attention_mask']
}

In [9]:
def gpt_loss(labels, logits):
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    # Masking to avoid calculating loss on padding tokens
    mask = tf.cast(tf.not_equal(labels, 0), tf.float32)
    loss = loss_fn(labels, logits, sample_weight=mask)
    return loss

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
              loss=gpt_loss)

history = model.fit(gpt_train_inputs,
                    label_train_tokenized['input_ids'],
                    epochs=2,
                    batch_size=8,
                    validation_data=(gpt_val_inputs, label_val_tokenized['input_ids']))

Epoch 1/2
625/625 [==============================] - 617s 895ms/step - loss: 3.4231 - val_loss: 3.2104
Epoch 2/2
625/625 [==============================] - 558s 892ms/step - loss: 3.2460 - val_loss: 3.1485


In [19]:
def generate_translations(model, tokenizer, text_data, max_length=100):
    translations = []

    for text in text_data:
        # Prepare the prompt
        input_text = "Translate English to Chinese: " + text
        # Tokenize input text
        input_ids = tokenizer.encode(input_text, return_tensors='tf')

        # Generate translation
        output_ids = input_ids
        for _ in range(max_length):
            outputs = model(output_ids)[0]
            # Get the token id for the last position
            next_token_id = tf.argmax(outputs[:, -1, :], axis=-1, output_type=tf.int32)
            next_token_id = tf.expand_dims(next_token_id, axis=0)
            # Concatenate with the previous output_ids
            output_ids = tf.concat([output_ids, next_token_id], axis=-1)

            # Check if the last token is eos_token
            if next_token_id == tokenizer.eos_token_id:
                break

        translation = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        translations.append(translation)

    return translations

# Generate translations for validation data
test_translations = generate_translations(model, tokenizer, test_examples_str)

In [24]:
test_examples_str

['Beyond showing no benefit from mammography screening, the Canadian trial highlighted one of the practice’s major disadvantages: over-diagnosis.',
 '(These and related issues were raised at a recent event on Capitol Hill in which I took part.)',
 'The world economy needs them – and no consumer-facing company can afford to ignore them.',
 'It used to be difficult to find anything to do on a Saturday night in Shanghai.',
 'Some preach that Muslims are destroying Western civilization.',
 'The first is the relationship between small and large economies.',
 "In other words, it fulfills Russia's vision of a federated Ukraine, whereby the Russian-oriented east would have a say in foreign affairs and security matters.",
 'Almost everywhere, these surviving indigenous populations became subservient members of European-led societies.',
 'Andy Lippman of the MIT Media Lab has a very good video explainer on this issue, which should be required viewing for policymakers (and voters).',
 'Our econom

In [21]:
from nltk.translate.bleu_score import corpus_bleu

# Prepare reference translations
# Here, each reference translation should be a list of possible translations
reference_translations = [[ref.split()] for ref in test_labels_str]  # Assuming val_labels_str is a list of reference translations
candidate_translations = [trans.split() for trans in test_translations]

# Calculate BLEU score
bleu_score = corpus_bleu(reference_translations, candidate_translations)
print(f'BLEU score: {bleu_score}')

BLEU score: 0


In [22]:
reference_translations

[[['加拿大试验不仅未能证明乳腺X光筛查的好处，还突出了上述方法的一个主要缺陷：诊断过度。']],
 [['（这些问题以及相关问题在最近的国会山会议上被提出，我也参与了这次会议。', '）']],
 [['世界经济需要他们', '—', '—', '任何面向消费者的公司可以忽视他们。']],
 [['以前在上海，星期六晚上无事可做。']],
 [['有人宣扬穆斯林正在摧毁西方文明。']],
 [['第一个关系是小经济体与大经济体之间的关系。']],
 [['换言之，它符合俄国的乌克兰“联邦”构想，亲俄的东部地区将拿到外交和安全事务发言权。']],
 [['但无论身在何处，这些活下来的土著人口都成了以欧洲人为首社会的附属成员。']],
 [['麻省理工学院媒体实验室的安迪·利普曼（Andy', 'Lippman）对这一问题进行了非常棒的视频解释，决策者（以及选民）都应该看看。']],
 [['我们的经济学家估算，将疟疾发病率降低50%的投资可以为社会带来35倍的回报。']],
 [['在印度和中国这样的高增长发展中国家，对全球化和数字技术的态度往往更加积极，这些国家的进步非常明显，数字技术看起来更像是增长引擎而不是威胁。']],
 [['诚然，一些国家已经正式宣布他们将逐渐退出或者放弃使用核能。',
   '这些国家包括比利时，意大利，德国，瑞士，以及秘鲁',
   '-',
   '-',
   '-唯一一个加入这项声明的非欧洲国家。']],
 [['大部分发达和新兴经济体的实体经济问题严重，但是，直到目前，金融市场仍然在央行加码宽松的情况下频创高峰。']],
 [['这种压制持续到1961年，赫鲁晓夫允许再次揭露斯大林时代的罪恶。']],
 [['2014年，内塔尼亚胡坚持要求巴勒斯坦承认以色列是犹太国家，成为压倒和平谈判的最后一根稻草。']],
 [['央行被广泛视为在本世纪最初的几年里玩忽职守。']],
 [['欧盟在这方面处于更佳地位，因为还不存在任何本地平台巨头。']],
 [['乍一看，这一要求相当合理。']],
 [['是的，当我们与合作伙伴共同创建中国第一家投资银行时，我们分享了自身的业务实践，专利产品和分销系统。']],
 [['如果企业投资委员会变得更为谨慎，那么我们希望美联储、欧洲央行、英格兰银行和其它银行会降低利率，增大贷款的吸引力。']]

In [23]:
candidate_translations

[['Translate',
  'English',
  'to',
  'Chinese:',
  'Beyond',
  'showing',
  'no',
  'benefit',
  'from',
  'mammography',
  'screening,',
  'the',
  'Canadian',
  'trial',
  'highlighted',
  'one',
  'of',
  'the',
  'practice’s',
  'major',
  'disadvantages:',
  'over-diagnosis.',
  'The',
  'study,',
  'published',
  'in',
  'the',
  'journal',
  'Cancer',
  'Epidemiology,',
  'Biomarkers',
  '&',
  'Prevention,',
  'found',
  'that',
  'women',
  'who',
  'received',
  'mammography',
  'screening',
  'were',
  'more',
  'likely',
  'to',
  'have',
  'a',
  'breast',
  'cancer',
  'diagnosis',
  'than',
  'women',
  'who',
  'did',
  'not.',
  'The',
  'study',
  'also',
  'found',
  'that',
  'women',
  'who',
  'received',
  'mammography',
  'screening',
  'were',
  'more',
  'likely',
  'to',
  'have',
  'a',
  'breast',
  'cancer',
  'diagnosis',
  'than',
  'women',
  'who',
  'did',
  'not.',
  'The',
  'study,',
  'published',
  'in',
  'the',
  'journal',
  'Cancer',
  'Epid

In [12]:
#save the model
from tensorflow.keras.utils import custom_object_scope

model_h5_path = "content/sample_data/save/"
model.save(model_h5_path)

In [15]:
from tensorflow.keras.models import load_model

model_h5_path = "content/sample_data/save/"
loaded_model = tf.keras.models.load_model(model_h5_path, custom_objects={'gpt_loss': gpt_loss})

In [ ]:
#1. Able to fune-tune the model and check out the BLEU score.
    #find the bug and rerun it.
#2. get acess to layers of transformer
#3. FFN intermediate